In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import datetime
import time
import urllib
import os
from tqdm import tqdm
import math
import requests
from astropy.time import Time
from scipy.optimize import curve_fit
import plotly.express as px
from bs4 import BeautifulSoup
import requests, lxml, re, json
import pickle
from umap import UMAP
from astropy.io import fits
import plotly.express as px
import sys
sys.path.insert(0,'../pygrb')
import warnings
warnings.filterwarnings('ignore')
from grb.light_curves import *
from grb.furie import *

Load Konus-wind catalog of GRB with known reshift

In [ ]:
with open('data/redshifts.pkl','rb') as f:
    redshifts = pickle.load(f)

detectors = {0:'n0',1:'n1',2:'n2',3:'n3',4:'n4',5:'n5',6:'n6',7:'n7',8:'n8',9:'n9',10:'na',11:'nb',12:'b0',13:'b1'}
with open('data/fermi_bursts_cat.txt','r') as f:
    gbm_cat = pd.DataFrame([line.split() for line in f],columns=['grb_name','date','time','t90','t90_start','grb_code','mask']).dropna()
    gbm_cat['redshift'] = gbm_cat['grb_code'].apply(lambda x:redshifts.get(x,None))
gbm_cat['mask'] = gbm_cat['mask'].str[:12]
gbm_cat = gbm_cat.set_index('grb_code')

Load light curve data using pygrb package

In [ ]:
def load_furie_data(redshifts, apply_redshift = True):
    pds = {}
    lcs = {}
    for code in redshifts:
        try:
            catalog_data = gbm_cat.loc[code]
        except KeyError:
            logging.error(f'{code} not found')
            continue
        try:
            lc = GBM_LightCurve.load(f'{code}_{apply_redshift}')
        except FileNotFoundError:
            if apply_redshift:
                lc = GBM_LightCurve(code,catalog_data['mask'],catalog_data['redshift'],filter_energy={'low_en':50,'high_en':300},apply_redshift=True)
            else:
                lc = GBM_LightCurve(code,catalog_data['mask'],catalog_data['redshift'])
            lc.save(f'{code}_{apply_redshift}')
        try:
            if apply_redshift:
                pds[code] = FurieLightCurve(lc,(float(catalog_data['t90_start'])/(1+redshifts[code]),(float(catalog_data['t90'])+float(catalog_data['t90_start']))/(1+redshifts[code])),pad_size=30000)
            else:
                pds[code] = FurieLightCurve(lc,(float(catalog_data['t90_start']),(float(catalog_data['t90'])+float(catalog_data['t90_start']))),pad_size=30000)
            lcs[code] = lc.rebin() # reset data
        except Exception as e:
            logging.error(e)
    return pds,lcs

In [ ]:
pds_i, lcs_i = load_furie_data(redshifts)
pds_o, lcs_o = load_furie_data(redshifts, apply_redshift=False)

Test calcultions on GRB 150301 and its background interval

In [ ]:
code = 'bn150301818'
lcs_i[code].set_intervals((lcs_i[code].times[0],float(gbm_cat.loc[code]['t90_start'])/(1+redshifts[code])-15)).plot()
np.mean(lcs_i[code].signal),np.var(lcs_i[code].signal)

In [ ]:
plt.plot(*make_pds(lcs_i[code].signal,0.01,None))
print(np.mean(make_pds(lcs_i[code].signal,0.01,None)[1]))
plt.axhline(2)
plt.xscale('log')
plt.yscale('log')

In [ ]:
# summed = []
# fig = plt.figure()
res = []
for code in lcs:
    if code not in ('bn150101641','bn100814160','bn090926181') :
        try:
            # lcs[code].set_intervals((lcs[code].times[0],min(float(catalog_data['t90_start'])/(1+redshifts[code]),0))).plot()
            # print(lcs[code].times[0],min(float(gbm_cat.loc[code]['t90_start'])/(1+redshifts[code]),0))
            times, data = limit_to_time_interval(lcs[code].times,
                                          lcs[code].signal,
                                          (lcs[code].times[0],float(gbm_cat.loc[code]['t90_start'])/(1+redshifts[code])-15))
            param = np.polyfit(times,data,1)
            data = data - np.polyval(param,times)
            # print(lcs[code].times.shape, data[1].shape, lcs[code].times[0], min(float(gbm_cat.loc[code]['t90_start'])/(1+redshifts[code]),0))
            res += list(make_pds(data+np.square(np.std(data)), 
                                                       lcs[code].resolution,
                                                       None)[1][make_pds(data+np.square(np.std(data)), 
                                                       lcs[code].resolution,
                                                       None)[0]>20])
            # summed.append([code,s,s_err,redshifts[code],float(gbm_cat.loc[code]['t90'])/(1+redshifts[code])])
            # plt.scatter(t,s)
            # plt.axhline(2)
            # plt.xscale('log')
            # plt.yscale('log')
        except Exception as e:
            pass

In [ ]:
plt.hist(res,bins=np.logspace(-2,2,30))
plt.axvline(2,color='red')
plt.axvline(np.median(res),color='black')
plt.xscale("log")
print(2,np.median(res))

In [ ]:
summed = []
for code in pds:
    if code not in ('bn150101641','bn100814160','bn090926181') :
        
        t,t_err,s,s_err = group_log_bins(pds[code].freqs,pds[code].ps,N_bins=30)
        summed.append([code,s,s_err,redshifts[code],float(gbm_cat.loc[code]['t90'])/(1+redshifts[code])])
        pds[code].plot()
        plt.axhline(2)
        # plt.title(code)
        # plt.show()

In [ ]:
res = pd.DataFrame(summed,columns=['code','vector','s_err','z','t90'])
res = res[res['vector'].apply(sum)>0]

plt.errorbar(t[2:],
            (np.sum(res['vector'].apply(lambda x:(np.asarray(x)-2)/np.sum(x)),axis=0))[2:]*np.power(t[2:],1.2),
            xerr = t_err[2:],
            yerr=np.sqrt(np.sum(res['s_err'].apply(lambda x:np.square(np.asarray(x)))/np.square(res['vector'].apply(lambda x:np.sum(x))),axis=0)/86)[2:]*np.power(t[2:],1.2),fmt='o')
plt.xscale('log')
plt.yscale('log')
plt.title(f"median z = {res['z'].median()}, median t90 = {res['t90'].median()}")

In [ ]:
res_2 = pd.DataFrame(summed,columns=['code','vector','s_err','z','t90'])
res_2 = res_2[res_2['vector'].apply(sum)>0]
res = res_2[res_2['z']<(res_2['z'].median()/1)]

plt.errorbar(t[2:],
            (np.sum(res['vector'].apply(lambda x:(np.asarray(x)-2)/np.sum(x)),axis=0))[2:]*np.power(t[2:],0.8),
            xerr = t_err[2:],
            yerr=np.sqrt(np.sum(res['s_err'].apply(lambda x:np.square(np.asarray(x)))/np.square(res['vector'].apply(lambda x:np.sum(x))),axis=0)/res.shape[0])[2:]*np.power(t[2:],0.8),fmt='o')
plt.xscale('log')
plt.yscale('log')
plt.title(f"median z = {res['z'].median()}, median t90 = {res['t90'].median()}")
# plt.ylim(6e-2,2e-1)

In [ ]:
res_2 = pd.DataFrame(summed,columns=['code','vector','s_err','z','t90'])
res_2 = res_2[res_2['vector'].apply(sum)>0]
res = res_2[res_2['z']>(res_2['z'].median()/1)]

plt.errorbar(t[2:],
            (np.sum(res['vector'].apply(lambda x:(np.asarray(x)-2)/np.sum(x)),axis=0))[2:]*np.power(t[2:],1.2),
            xerr = t_err[2:],
            yerr=np.sqrt(np.sum(res['s_err'].apply(lambda x:np.square(np.asarray(x)))/np.square(res['vector'].apply(lambda x:np.sum(x))),axis=0)/res.shape[0])[2:]*np.power(t[2:],1.2),fmt='o')
plt.xscale('log')
plt.yscale('log')
plt.title(f"median z = {res['z'].median()}, median t90 = {res['t90'].median()}")

In [ ]:
res = pd.DataFrame(summed,columns=['code','vector','s_err','z'])
res = res[res['vector'].apply(sum)>0]

plt.errorbar(t[2:],np.sum(res['vector'].apply(lambda x:np.asarray(x)-2),axis=0)[2:]*np.power(t[2:],1.6),yerr = np.sqrt(np.sum(res['s_err'].apply(lambda x:np.square(np.asarray(x))),axis=0)/86)[2:]*np.power(t[2:],1.6),xerr = t_err[2:],fmt='o')
plt.xscale('log')
plt.yscale('log')

In [ ]:
bkg_data = {}
for code in redshifts:
    try:
        catalog_data = gbm_cat.loc[code]
    except KeyError:
        print(code,'not found')
        continue
    try:
        lc = GBM_LightCurve.load(code)
    except FileNotFoundError:
        lc = GBM_LightCurve(code,catalog_data['mask'],catalog_data['redshift'],save_photons=True,filter_energy=False)
        lc.save(code)
    dat = lc.photon_data[(lc.photon_data[:,0]<float(catalog_data['t90_start']))|(lc.photon_data[:,0]>float(catalog_data['t90_start'])+float(catalog_data['t90']))]
    counts, bins = np.histogram(dat[:,1],np.logspace(0,4,100))
    bkg_data[code] = counts
sumed_2 = np.sum([bkg_data[code] for code in bkg_data if code != 'bn090516353'],axis=0)

In [ ]:
limit = 0.61
substracted = sumed * (np.max(sumed_2)/np.max(sumed)) - sumed_2 *limit
cs = np.cumsum(substracted)
plt.stairs(sumed_2/np.max(sumed_2)*limit, bins,label='Background, normalized and rescaled')
plt.stairs(sumed/np.max(sumed), bins, label = 'Events, normalized')
plt.stairs(sumed/np.max(sumed)-sumed_2/np.max(sumed_2)*limit, bins, label = 'Events, substracted')
plt.xscale('log')
plt.ylim(-0.2,1.4)
plt.legend(loc='upper left')
plt.axvline(50,color='b',linestyle='--')
plt.axvline(300,color='b',linestyle='--')
plt.xlabel('Photon energy (keV)')
# plt.axvline(bins[np.searchsorted(cs, np.percentile(cs, 94))],color='b')
# plt.axvline(bins[np.searchsorted(cs, np.percentile(cs, 22))],color='b')

In [ ]:
substracted

Один всплеск не найден, скорее всего code изменился на +- 1 (из-за обновления точности каталога)

In [ ]:
lc = SPI_ACS_LightCurve('2021-06-19 23:59:25', 300)

In [ ]:
lc = GBM_LightCurve('bn210619999','00001000100000')

In [ ]:
lc = GBM_LightCurve('bn210619999','00001000100000', data = pd.read_csv('test.csv')[['times','signal']].values)

In [ ]:
lc = GBM_LightCurve.load('kek')

In [ ]:
flc = FurieLightCurve(lc,(0.576-15,54.785+60))

In [ ]:
flc.plot(N_bins=None,kind='plot')
flc.plot(N_bins=20,kind='errorbar',color='black')
plt.axhline(2,color='r')